# Dance-Dance-Revolution

Динамическое программирование часто помогает решить различные задачи оптимизации. В предыдущей лекции мы рассмотрели случай, когда необходимо было найти наименьшее число операций для превращения одного слова в другое. В этот раз разберем еще один пример задачи, в котором необходимо найти минимум.

В далеком 1998 году, Японская компания выпустила аркадный игровой автомат, который невероятно быстро завоевал популярность и любовь огромного числа игроков. Суть ее заключалась в следующем:

Вся игра происходит на танцевальной платформе, на которой есть 4 панели: "Лево", "Право", "Вверх", "Вниз" (‘L’, ‘R’, ‘U’ и ‘D’ соответственно). 

<img src="img/DDR1.png">

Во время игры играет мелодия, условно разбитая на N позиций в такт музыке. Для каждой такой позции указано, какие стрелки должны быть нажаты во время воспроизведения этой позиции - или одна или две одноверемнно нажатые стрелки.

Наша цель в этой игре - нажать на все стрелки в требуемой последовательности при этом минимизировав расстояние, которое придется сделать ногами.

Некоторые уточнения: игрок имеет право стоять на стрелке даже тогда, когда не требуется ее наживать - необходимо лишь, чтобы при этом были нажаты те, которые указывает игра.
Пройденное расстояние подразумевается евклидовым, то есть расстояние между центральной панелью и любой другой - 1, а между верхней и левой, например, $ \sqrt{2}$

Изначальное положение ног - обе стоят на центральной. Финальное положение ног - любое.

Для того, чтобы подступиться к этой задаче, сразу заметим некоторые факты, на которым мы и построим итоговое решение.

* На каждой игровой позиции возможно всего $5 \cdot 5 = 25$ конфигураций (положений ног) - 5 способов поставить правую ногу и пять способов поставить левую ногу (независимо).

* Путь для каждой конфигурации на i-той игровой позиции мы знаем минимальное расстояние, которое необходимо проделать, чтобы с начальной 0-ой игровой позиции перейти в текущую на i-том. Тогда мы может посчитать такое расстояние для каждой конфигурации уже на i+1-ой игровой позиции:

 $d_{i+1}[A][B] = min \{ d_i[C][D] + cost( C \cdot D \rightarrow A \cdot B ) \:\:\:\:\: \forall C, D \in \{ Right, Left, Up, Down, Center \} \}$
 
И правда, для этого всего лишь нужно выбрать такую конфигурацию на предыдущей игровой позиции, чтобы расстояния перехода в нее + расстояние для перехода из нее в текущую была минимальна. Это суммарное расстрояние и будет искомым значением.

* Рассчитывать минимальное расстояние на i-той игровой позиции необходимо только для тех конфигураций, которые допускаются игрой. 

* Минимальное значение из подходящий конфигураций на n-той игровой позиции и есть ответ, так как это и будет минимальное расстояние, которое необходимо проделать начиная с 0-й позиции передвигая ноги только так, как это допускает игра до последней n-той позиции.

Таким образом решение данной задачи будет состоять в вычислении минимальных расстояний для допустимых конфигураций для каждой игровой позиции. Минимальное число на последней игровой позиции - ответ к задаче.

In [1]:
#include <climits>
#include <vector>
#include <string>
#include <iostream>
#include <fstream>
#include <cmath>

In [2]:
double play_DDR(std::string gameFile) {
    std::ifstream gf(gameFile);

    const int SIZE = 5; // всего 5 плиток
    int x[SIZE] = {-1, 0, 0, 0, 1}; // координаты плиток
    int y[SIZE] = {0, -1, 0, 1, 0}; // кодирование: 0 - Left, 1 - Down, 2 - Center, 3 - Up, 4 - Right
    auto dist = [&x, &y](int i, int j) {
        return sqrt( (x[i]-x[j])*(x[i]-x[j]) + (y[i]-y[j])*(y[i]-y[j]) ); // евклидово расстояние между
    };
    int N;
    gf >> N;
    std::vector< std::vector< std::vector<double> > > E(N+1, std::vector< std::vector<double> >(5, std::vector<double>(5, INFINITY)));
    // E[i][A][B] - стоимость(расстояние) конфигцрации, при котогой правая нога стоит на A, левая на B на i-той игровой позиции
    // изначально положим бесконечность в каждую конфигурацию
    std::vector< std::vector<bool> > game(N+1, std::vector<bool>(5, false));
    // данные игры. game[i][C] == True -> на i-той позиции нога должна находиться на С, False -> не обязана
    for(int i = 0; i < N; i++) {
        int q;
        gf >> q;
        for(int j = 0; j < q; j++) { // считывает параметры игры
            char position;
            gf >> position;
            switch (position) {
                case 'U':
                    game[i][3] = true;
                    break;
                case 'D':
                    game[i][1] = true;
                    break;
                case 'L':
                    game[i][0] = true;
                    break;
                case 'R':
                    game[i][4] = true;
                    break;
            }
        }
    }

    E[0][2][2] = 0.0; // Изначально стоим в центре. Расстояние равно 0
    for(int i = 0; i < N; i++) { // начинаем расчитывать расстояния конфигураций
        for(int C = 0; C < SIZE; C++) {
            for(int D = 0; D < SIZE; D++) { // Перебираем все конфигурации на текущей позиции
                bool incorrect = false;
                for(int game_conf = 0; game_conf < SIZE; game_conf++) {
                    if(game[i][game_conf] and !( game_conf == C or game_conf == D ) ){ // проверяем, является ли данная конфигурация допустимой с точки зрения игры
                        incorrect = true;
                    }
                }
                if(incorrect) continue;  // пропускаем все неподходящие конфигурации

                for(int A = 0; A < SIZE; A++){
                    for(int B = 0; B < SIZE; B++) { // начитаем вычисление каждой конфигурации на следующей позиции
                        double cost = std::min(dist(A, C) + dist(B, D), // у нас есть только два варианта смены конфигцрации
                                               dist(A, D) + dist(B, C)); // или A->C и B->D или наоборот
                        E[i+1][C][D] = std::min(E[i+1][C][D], E[i][A][B]+cost); // выбираем минимальный путь
                    }
                }
            }
        }
    }

    double result = INFINITY;
    for(int A = 0; A < SIZE; A++) {
        for(int B = 0; B < SIZE; B++) {
            result = std::min(result, E[N][A][B]); // Выбираем минимальный на последней позиции
        }
    }
    return result;
}

In [3]:
void print_file(std::string f) { // печать файла для наглядности
    std::ifstream file(f);
    std::string s;
    while(std::getline(file, s)) {
        std::cout << s << std::endl;
    }
    
}

Примечание: файлы игры имеют следующую струкруру
* Вначале идет число N- количество игровый позиций
* Потом идет N строк в каждой из которых описывается соответствующая позиция
 * Вначале указывается 1 или 2 клавишы должны быть нажаты
 * После соответствующее количество клавиш, которые должны быть нажаты

In [9]:
print_file("files/1");
std::cout << "-------" << std::endl;
std::cout << play_DDR("files/1");

3
1 U
1 D
2 L R
-------
4.82843

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fae043086e0


В данной игре игрок правую и левую ногу поставил на U и D изначально, а потом сдвинул их на L и R. Итого он затратил $ 1 + 1 + \sqrt{2} + \sqrt{2} \approx 4.82843$ 

In [11]:
print_file("files/2");
std::cout << "-------" << std::endl;
std::cout << play_DDR("files/2");

2
1 U
1 U
-------
1

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fae043086e0


Здесь игроку пришлось просто передвинуть одну ногу на U и все.

In [12]:
print_file("files/3");
std::cout << "-------" << std::endl;
std::cout << play_DDR("files/3");

3
2 R L
2 U R
2 U R
-------
3.41421

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fae043086e0


In [13]:
print_file("files/4");
std::cout << "-------" << std::endl;
std::cout << play_DDR("files/4");

19
2 R L
2 U R
2 U R
2 R L
2 R U
2 U R
2 L D
2 R U
2 R L
2 U R
2 U L
2 U R
2 L U
2 L R
2 U L
2 L U
2 U L
2 L D
2 R L
-------
26.9706

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fae043086e0


In [14]:
print_file("files/5");
std::cout << "-------" << std::endl;
std::cout << play_DDR("files/5");

20
2 U R
1 L
1 U
2 U R
1 L
1 U
2 U R
1 L
1 U
2 U R
1 L
1 U
2 U R
1 L
1 U
2 U R
1 L
1 U
2 U R
1 L
-------
20.3848

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fae043086e0
